# Exploring Convolutional Layers Through Data and Experiments


## SetUp

In [1]:
%pip install numpy matplotlib tensorflow

  Using cached numpy-2.4.2-cp314-cp314-win_amd64.whl.metadata (6.6 kB)
  Using cached matplotlib-3.10.8-cp314-cp314-win_amd64.whl.metadata (52 kB)
Note: you may need to restart the kernel to use updated packages.


ERROR: Ignored the following versions that require a different python version: 1.21.2 Requires-Python >=3.7,<3.11; 1.21.3 Requires-Python >=3.7,<3.11; 1.21.4 Requires-Python >=3.7,<3.11; 1.21.5 Requires-Python >=3.7,<3.11; 1.21.6 Requires-Python >=3.7,<3.11; 1.26.0 Requires-Python >=3.9,<3.13; 1.26.1 Requires-Python >=3.9,<3.13
ERROR: Could not find a version that satisfies the requirement tensorflow (from versions: none)
ERROR: No matching distribution found for tensorflow


## Dataset Load

In [2]:
from tensorflow.keras.datasets import cifar100

# Cargar dataset
(x_train, y_train), (x_test, y_test) = cifar100.load_data()

print(x_train.shape)  # (50000, 32, 32, 3)
print(y_train.shape)  # (50000, 1)


ModuleNotFoundError: No module named 'tensorflow'